# TopBrass Profit Optimization with GAMSpy

This notebook solves a simple linear programming problem using `GAMSpy`. The goal is to find the optimal amount of football and soccer trophies to produce to maximize profits.

## Problem
* **Football Trophies** (var `f`) are larger in size so they require 4 sq. feet of wood on their base and yield a profit of $12
* **Soccer Trophies** (var `s`) are smaller so they need 2 sq. feet of wood on the base and yield a smaller profit of $9.

## Constraints
1.  **Football Inventory:** Maximum of 1,000 units.
2.  **Soccer Inventory:** Maximum of 1,500 units.
3.  **Plaques:** 1,750 total available (one per trophy).
4.  **Wood:** 4,800 sq. feet total available.

**Objective:** Find the production of football trophies and soccer trophies that will yield the most profit for TopBrass

# Abstract Model Formulation

To make the model scalable and applicable to more complex problems (e.g., thousands of products and resources), we can define it in abstract mathematical terms.

----------

| Symbol | Description | Example |
| :--- | :--- | :--- |
| **Sets** | | |
| $I$ | The set of all products (items). | `{'football', 'soccer'}` |
| $R$ | The set of all resources. | `{'plaques', 'wood'}` |
| **Parameters** | | |
| $c_i$ | Profit per unit of product $i \in I$. | $c_{football} = 12$ |
| $u_i$ | Upper production limit (inventory) for product $i \in I$. | $u_{football} = 1000$ |
| $b_r$ | Available amount of a given resource $r \in R$. | $b_{wood} = 4800$ |
| $a_{ri}$ | Amount of resource $r$ required to produce one unit of product $i$. | $a_{wood, football} = 4$ |
| **Variable** | | |
| $x_i$ | **Decision Variable:** The number of units of product $i \in I$ to produce. | This is what we want to find. |

----------

## Mathematical Model

The abstract problem can be formulated with the following equations:

**Objective Function:**
The goal is to maximize the total profit.
$$ \max \sum_{i \in I}{c_ix_i}$$

**Constraints:**
1.  **Resource Limits:** For each resource, the total amount used cannot exceed the available amount.
    $$\sum_{i \in I}{a_{ri} x_i} \leq b_r, \forall_r \in R$$
2.  **Production Limits:** The number of units produced cannot exceed the inventory limits.
    $$ x_i \leq u_i, \forall_i \in I$$
3.  **Non-Negativity:** It is not possible to produce a negative number of trophies.
    $$ x_i \geq 0, \forall_i \in I$$

In [39]:
# Imports
import gamspy as gp
import numpy as np

## Step 1. Set up the Model

In [ ]:
# GAMSpy containers act as central repositories for model components
m = gp.Container()

# Use sets to represent products and resources 
i = gp.Set(m, 'i', records = {'football', 'soccer'})
r = gp.Set(m, 'r', records = {'plaques', 'wood'})

# use parameters for constants
u = gp.Parameter(m, 'u', domain = [i], records=np.array([1000, 1500]), description='Inventory limits')
b = gp.Parameter(m, 'b', domain=[r], records=np.array([1750, 4800]), description='resource limitations')
a = gp.Parameter(m, 'a', domain=[r,i], records=np.array([[1, 1], [4, 2]]), description='per-unit recourse calculations')
c = gp.Parameter(m, 'c', domain=[i], records=np.array([12, 9]), description='profits')

In [ ]:
# Define the decision variable
x = gp.Variable(m, 'x', 'positive', domain = [i], description = 'trophies')
x.up[i] = u[i]

In [ ]:
# write model's data to a GAMS Data eXchange (GDX) file for debugging or export
# m.write('tbdata.gdx')

In [ ]:
# Define the Constraints as Equations
res_lim = gp.Equation(m, 'res_lim', domain=[r], description='resource limit')
# Define the logic for the constraint: For each resource 'r', the sum of units produced (x[i]) multiplied by their resource consumption (a[r,i])
# must be less than or equal to the available amount of that resource (b[r])
res_lim[r] = gp.Sum(i, a[r,i]*x[i]) <= b[r]

## Step. 2: Solving the Model

In [44]:
topBrass = gp.Model(
    m,
    name= 'topBrass',
    equations= m.getEquations(),
    objective= gp.Sum(i,c[i]*x[i]),
    sense = 'max',
    problem = 'lp',)


In [45]:
topBrass.solve()

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,17700.0,3,3,LP,CPLEX,0.0


### IMPORTANT

Now that I'm using .up, the upper value is no longer inf. Also notice that the variables are presented like databases.

In [46]:
display(x.records)

,i,level,marginal,lower,upper,scale
0,soccer,650.0,0.0,0.0,1000.0,1.0
1,football,1100.0,0.0,0.0,1500.0,1.0
